In [1]:
import sys
sys.path.append("../Modules/")
sys.path.append("../")
import os

import pandas as pd
import numpy as np

from configobj import ConfigObj
from configobj import flatten_errors
from validate import Validator
    
import logging

#fmt = 'logging.Formatter(''%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
fmt = '%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
logging.basicConfig(level=logging.INFO, format=fmt)
logger = logging.getLogger(__name__)


In [2]:
fileTimeList = [];
[fileTimeList.append(name) for name in os.listdir("../Data/") if name[0] is not '.'];
fileTimeList

['2018-4-30-13-49',
 '2018-4-26-11-31',
 '2018-4-27-12-32',
 '2018-4-30-15-4',
 '2018-4-26-15-49',
 '2018-4-27-11-16',
 '2018-4-26-12-30',
 '2018-4-27-13-41']

In [3]:
# fileTimeList = [fileTimeList[1]]
# fileTimeList

In [ ]:
fileTimeList[0]

'2018-4-30-13-49'

In [ ]:
from analysisParameters import loadParameters
from loadDataE1 import loadSessionDict

from processDataE1 import projectTrajectories, vectorMovementModel, calcCatchingError, calcTrackingError
sessionFiles = []

subNum = 0
fileName = fileTimeList[0]
#for subNum, fileName in enumerate(fileTimeList):

analysisParameters = loadParameters(fileName)

# set loadProcessedData to True if you want to reprocess the file.  
sessionDict = loadSessionDict(analysisParameters,
                              loadParsedData = False,
                              loadProcessedData = False,
                                )

sessionDict['trialInfo']['subjectNumber'] = subNum


INFO_loadDataE1-loadSessionDict(): - Parsing raw data: exp_data-2018-4-30-13-49 ***


In [ ]:
# sessionDict['trialInfo']['trialNumber']
#sessionDict['raw'].eventFlag 
a = sessionDict['raw'].eventFlag.shift(-1)
a.iloc[-1] = 'trialEnd'

In [ ]:
expansionGain_tr = [np.nan] * len(sessionDict['trialInfo'])
passingLocX_tr = [np.nan] * len(sessionDict['trialInfo'])

for (blockNum,trialNum), trialData in sessionDict['processed'].groupby(('blockNumber','trialNumber')):

    tInfoIloc = np.intersect1d(np.where(sessionDict['trialInfo']['blockNumber']==blockNum),
                           np.where(sessionDict['trialInfo']['trialNumber']==trialNum))[0]

    tt = trialData.trialType.iloc[1]
    expansionGain_tr[tInfoIloc] = sessionDict['expConfig']['trialTypes'][tt]['expansionGain']
    passingLocX_tr[tInfoIloc] = sessionDict['expConfig']['trialTypes'][tt]['passingLocNormX']

sessionDict['trialInfo']['expansionGain'] = np.array(expansionGain_tr,dtype=np.float)
sessionDict['trialInfo']['passingLocX'] = np.array(passingLocX_tr,dtype=np.float)

logger.info('Added sessionDict[\'trialInfo\'][\'expansionGain\']')
logger.info('Added sessionDict[\'trialInfo\'][\'passingLocX\']')

In [ ]:
#def calcTrackingError(sessionDict):

from processDataE1 import findAnalysisWindow
meanEyeToBallEdgeAz_tr = [np.nan] * len(sessionDict['trialInfo'])
meanEyeToBallCenterAz_tr = [np.nan] * len(sessionDict['trialInfo'])

meanEyeToBallEdge_tr = [np.nan] * len(sessionDict['trialInfo'])
meanEyeToBallCenter_tr = [np.nan] * len(sessionDict['trialInfo'])

ballChangeInRadiusDegs_tr = [np.nan] * len(sessionDict['trialInfo'])

for (blockNum,trialNum), trialData in sessionDict['processed'].groupby(('blockNumber','trialNumber')):

    tInfoIloc = np.intersect1d(np.where(sessionDict['trialInfo']['blockNumber']==blockNum),
                           np.where(sessionDict['trialInfo']['trialNumber']==trialNum))[0]

    def calcEyeToBallCenter(row):
        # The difference along az and el
        azimuthalDist = row['cycGIW_az'] - row['ball_az']
        elevationDist = row['cycGIW_el'] - row['ball_el']
        return (azimuthalDist,elevationDist)

    def calcEyeToBallEdge(row):
        # The difference along az and el
        azimuthalDist = row['cycGIW_az'] - row['ball_az']
        elevationDist = row['cycGIW_el'] - row['ball_el']

        ballRadiusDegs = np.float(row['ballRadiusDegs'])

        if np.float(azimuthalDist) > 0:
            azimuthalDist -= ballRadiusDegs
        else:
            azimuthalDist += ballRadiusDegs

        return (azimuthalDist,elevationDist)

    (startFr, endFr) = findAnalysisWindow(trialData)
    endFr = endFr-1

    absAzEl_XYZ = sessionDict['processed'].iloc[startFr:endFr].apply(lambda row: calcEyeToBallCenter(row), axis=1)
    (azDist,elDist) = zip(*absAzEl_XYZ)

    meanEyeToBallCenterAz_tr[tInfoIloc] = np.mean(azDist)
    meanEyeToBallCenter_tr[tInfoIloc] = np.nanmean([np.sqrt(np.float(fr[0]*fr[0] + fr[1]* fr[1])) for fr in absAzEl_XYZ ])

    absAzEl_XYZ = sessionDict['processed'].iloc[startFr:endFr].apply(lambda row: calcEyeToBallEdge(row), axis=1)
    (azDist,elDist) = zip(*absAzEl_XYZ)
    meanEyeToBallEdgeAz_tr[tInfoIloc] = np.mean(azDist)
    meanEyeToBallEdge_tr[tInfoIloc] = np.nanmean([np.sqrt(np.float(fr[0]*fr[0] + fr[1]* fr[1])) for fr in absAzEl_XYZ ])

    radiusAtStart = sessionDict['processed']['ballRadiusDegs'].iloc[startFr]
    radiusAtEnd = sessionDict['processed']['ballRadiusDegs'].iloc[endFr]
    ballChangeInRadiusDegs_tr[tInfoIloc] = radiusAtEnd - radiusAtStart

### Outside trial loop
sessionDict['trialInfo']['meanEyeToBallCenterAz'] = meanEyeToBallCenterAz_tr
sessionDict['trialInfo']['meanEyeToBallEdgeAz'] = meanEyeToBallEdgeAz_tr

sessionDict['trialInfo']['meanEyeToBallCenter'] = meanEyeToBallCenter_tr
sessionDict['trialInfo']['meanEyeToBallEdge'] = meanEyeToBallEdge_tr

sessionDict['trialInfo']['ballChangeInRadiusDegs'] = ballChangeInRadiusDegs_tr


In [ ]:
#sessionDict['raw'].eventFlag = sessionDict['raw'].eventFlag.shift(-1)

#sessionDict['raw'].groupby(['trialNumber','blockNumber']).get_group((1,0))
#sessionDict['processed'].groupby(('blockNumber','trialNumber')).groups.keys()
#sessionDict['raw'].groupby(['blockNumber','trialNumber']).get_group((1,2)).eventFlag
# Series.shift(periods=1, freq=None, axis=0, fill_value=None)[source]¶


In [ ]:
sessionDict['trialInfo']

In [ ]:
absAzEl_XYZ